In [9]:
import seaborn as sns

# Load Titanic dataset from seaborn
data = sns.load_dataset('titanic')

# print head
data.head
data.columns



Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')